In [ ]:
import pygame
import random
import math

# Initialize Pygame and set up the display
pygame.init()
screen = pygame.display.set_mode((800, 600))
pygame.display.set_caption("Autonomous Driving Simulation")

# Define colors
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
GRAY = (200, 200, 200)
DARK_GRAY = (50, 50, 50)
YELLOW = (255, 255, 0)

# Car properties
car_size = (30, 15)
car_max_speed = 3

# Traffic light properties
traffic_lights = {
    "north": {"pos": (400, 150), "state": "red"},
    "south": {"pos": (400, 450), "state": "red"},
    "east": {"pos": (550, 300), "state": "red"},
    "west": {"pos": (250, 300), "state": "red"},
}
traffic_light_order = ["north", "east", "south", "west"]
current_light = 0
light_timer = 0

# Road layout for a 4-way intersection with dividers and sidewalks
lanes = {
    "north": [(370, 0, 30, 150)],  # Left lane for northbound traffic
    "south": [(430, 450, 30, 150)],  # Left lane for southbound traffic
    "east": [(450, 280, 150, 30)],  # Left lane for eastbound traffic
    "west": [(200, 320, 150, 30)],  # Left lane for westbound traffic
}

# Sidewalks and dividers
sidewalks = [
    (300, 0, 100, 150),
    (300, 450, 100, 150),
    (0, 250, 150, 100),
    (650, 250, 150, 100),
]
dividers = [(390, 0, 20, 600), (0, 290, 800, 20)]

# Initialize car lists for each lane
cars = {"north": [], "south": [], "east": [], "west": []}


# Draw the environment (roads, cars, traffic lights)
def draw_environment():
    screen.fill(WHITE)

    # Draw sidewalks
    for sidewalk in sidewalks:
        pygame.draw.rect(screen, GRAY, sidewalk)

    # Draw lanes
    for lane in lanes["north"] + lanes["south"] + lanes["east"] + lanes["west"]:
        pygame.draw.rect(screen, DARK_GRAY, lane)

    # Draw dividers
    for divider in dividers:
        pygame.draw.rect(screen, YELLOW, divider)

    # Draw traffic lights with stop lines
    for light, data in traffic_lights.items():
        color = RED if data["state"] == "red" else GREEN
        pygame.draw.circle(screen, color, data["pos"], 20)
        if light == "north":
            pygame.draw.line(screen, WHITE, (350, 170), (450, 170), 5)
        elif light == "south":
            pygame.draw.line(screen, WHITE, (350, 430), (450, 430), 5)
        elif light == "east":
            pygame.draw.line(screen, WHITE, (530, 250), (530, 350), 5)
        elif light == "west":
            pygame.draw.line(screen, WHITE, (270, 250), (270, 350), 5)

    # Draw cars
    for direction, car_list in cars.items():
        for car in car_list:
            pygame.draw.rect(screen, BLUE, (*car["pos"], *car_size))

    pygame.display.flip()


# Update traffic light state every 200 frames
def update_traffic_lights():#Keeps the track of time
    global current_light, light_timer
    light_timer += 1
    if light_timer > 200:
        traffic_lights[traffic_light_order[current_light]]["state"] = "red"
        current_light = (current_light + 1) % 4
        traffic_lights[traffic_light_order[current_light]]["state"] = "green"
        light_timer = 0


# Spawn cars in each lane at random intervals
def spawn_cars():
    spawn_chance = 0.02  # Chance to spawn a new car per lane per frame
    directions = ["north", "south", "east", "west"]
    spawn_positions = {
        "north": [370, 600],
        "south": [430, 0],
        "east": [0, 280],
        "west": [800, 320],
    }
    for direction in directions:
        if random.random() < spawn_chance:
            new_car = {
                "pos": spawn_positions[direction][:],
                "dir": direction,
                "speed": car_max_speed,
            }
            cars[direction].append(new_car)


# Move cars with basic traffic light obedience and left-side driving
def move_cars(): #moving the cars based on current state of traffic lights
    for direction, car_list in cars.items():
        for car in car_list:
            # Move cars based on direction, staying on the left lane
            if car["dir"] == "north" and (
                traffic_lights["north"]["state"] == "green" or car["pos"][1] < 170
            ):
                car["pos"][1] -= car["speed"]
            elif car["dir"] == "south" and (
                traffic_lights["south"]["state"] == "green" or car["pos"][1] > 430
            ):
                car["pos"][1] += car["speed"]
            elif car["dir"] == "east" and (
                traffic_lights["east"]["state"] == "green" or car["pos"][0] > 270
            ):
                car["pos"][0] += car["speed"]
            elif car["dir"] == "west" and (
                traffic_lights["west"]["state"] == "green" or car["pos"][0] < 530
            ):
                car["pos"][0] -= car["speed"]

        # Remove cars that have exited the screen
        cars[direction] = [
            car
            for car in car_list
            if 0 <= car["pos"][0] <= 800 and 0 <= car["pos"][1] <= 600
        ]


# Simulation loop
running = True #flag to keep simulation running 
while running: #starting the main loop
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_n:
                traffic_lights[traffic_light_order[current_light]]["state"] = "red"
                current_light = (current_light + 1) % 4 #moves to next traffic light in cyclic manner
                traffic_lights[traffic_light_order[current_light]]["state"] = "green"
                light_timer = 0

    # Update traffic lights and spawn/move cars
    update_traffic_lights()
    spawn_cars()
    move_cars()

    # Draw the updated environment
    draw_environment()

    # Cap the frame rate
    pygame.time.Clock().tick(30) # clock.tick(30) ensures your game runs at a predictable speed

pygame.quit()

In [3]:
pip install pygame



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade pip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 656.8 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2
Note: you may need to restart the kernel to use updated packages.
